# Imports and Setup

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
from dotenv import load_dotenv
from graphdatascience import GraphDataScience
from langchain_community.graphs import Neo4jGraph

# Load Environment Variables, Initialize Neo4j Graph and Graph Data Science

In [ ]:
load_dotenv(".env", override=True)
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = os.getenv("NEO4J_DATABASE") or "neo4j"

kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE,
    enhanced_schema=False,
    refresh_schema=False,
)

gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

# Graph Projection
In order for any algorithm in the GDS library to run, we must first project a graph to run on. The graph is projected as a named graph.

In [ ]:
graphs = gds.run_cypher(
    """CALL gds.graph.list() YIELD graphName
RETURN graphName"""
)["graphName"].tolist()

if "myGraph" not in graphs:
    labels_list = kg.query(
        """CALL db.labels() YIELD label
        RETURN label"""
    )
    labels = [l["label"] for l in labels_list]

    relTypes_list = kg.query(
        """CALL db.relationshipTypes() YIELD relationshipType
        RETURN relationshipType"""
    )
    relTypes = [r["relationshipType"] for r in relTypes_list]

    graph, pd_series = gds.graph.project("myGraph", labels, relTypes)

    print("Created: 'myGraph'")

# Drop Graph Projection (Commented Out)

In [ ]:
# gds.run_cypher(f"CALL gds.graph.drop('myGraph') YIELD graphName")

# Visualisation Settings

In [ ]:
def plot_top_nodes(df, score_col="Score", title="Top Nodes", format="svg"):
    plt.figure(figsize=(10, 6))
    y_pos = np.arange(len(df))
    plt.barh(y_pos, df[score_col], align="center", alpha=0.95, color="tab:blue")
    rows = [
        f"{name} ({', '.join(labels)})"
        for name, labels in zip(df["Name"], df["Labels"])
    ]
    plt.yticks(y_pos, rows)
    plt.xlabel(score_col)
    plt.title(title)
    plt.gca().invert_yaxis()
    plt.tight_layout()
    plt.savefig(
        f"{title.lower().replace(' ', '_')}.{format}",
        dpi=300,
        bbox_inches="tight",
        format=format,
    )
    plt.show()

# Node Labels and Relationship Types Distribution

In [ ]:
labels_dist = gds.run_cypher(
    """MATCH (n) 
RETURN labels(n) AS Labels, count(*) AS Count 
ORDER BY Count DESC"""
)

labels_dist

In [ ]:
# labels_dist["Labels_Str"] = labels_dist["Labels"].apply(lambda x: ", ".join(x))

# plt.figure(figsize=(max(10, len(labels_dist) * 0.4), 6))
# y_pos = np.arange(len(labels_dist))
# plt.barh(y_pos, labels_dist["Count"], align="center", alpha=0.95, color="tab:blue")
# plt.yticks(y_pos, labels_dist["Labels_Str"])
# plt.xlabel("Node Count")
# plt.title("Node Labels Distribution")
# plt.gca().invert_yaxis()
# plt.tight_layout()
# plt.show()

In [ ]:
labels_dist.to_csv(r"labels_dist.csv", index=False, encoding="utf-8")

In [ ]:
rel_dist = gds.run_cypher(
    """MATCH ()-[r]->() 
RETURN type(r) AS RelationshipType, count(*) AS Count 
ORDER BY Count DESC"""
)

rel_dist

In [ ]:
# plt.figure(figsize=(10, 6))
# y_pos = np.arange(len(rel_dist))
# plt.barh(y_pos, rel_dist["Count"], align="center", alpha=0.95, color="tab:blue")
# plt.yticks(y_pos, rel_dist["RelationshipType"])
# plt.xlabel("Relationship Count")
# plt.title("Relationship Types Distribution")
# plt.gca().invert_yaxis()
# plt.tight_layout()
# plt.show()

In [ ]:
rel_dist.to_csv(r"rel_dist.csv", index=False, encoding="utf-8")

In [ ]:
missing = gds.run_cypher(
    """MATCH (n)
UNWIND keys(n) AS propertyKey
WITH propertyKey, n[propertyKey] AS propertyValue
WITH 
  count(*) AS totalProperties,
  count(CASE WHEN propertyValue = 'N/A' THEN 1 END) AS missingProperties
RETURN 
  totalProperties AS TotalProperties,
  missingProperties AS MissingProperties,
  round(100.0 * missingProperties / totalProperties, 2) AS MissingPercentage"""
)

missing

In [ ]:
missing.to_csv(r"missing.csv", index=False, encoding="utf-8")

# Betweenness Centrality
Betweenness centrality is a way of detecting the amount of influence a node has over the flow of information in a graph. It is often used to find nodes that serve as a bridge from one part of a graph to another.

The algorithm calculates shortest paths between all pairs of nodes in a graph. Each node receives a score, based on the number of shortest paths that pass through the node. Nodes that more frequently lie on shortest paths between other nodes will have higher betweenness centrality scores.

Betweenness centrality is implemented for graphs without weights or with positive weights. The GDS implementation is based on Brandes' approximate algorithm for unweighted graphs. For weighted graphs, multiple concurrent Dijkstra algorithms are used. The implementation requires O(n + m) space and runs in O(n * m) time, where n is the number of nodes and m the number of relationships in the graph.

# Stream
In the stream execution mode, the algorithm returns the centrality for each node.

In [ ]:
betweenness_stream = gds.run_cypher(
    """CALL gds.betweenness.stream('myGraph')
YIELD nodeId, score
WITH nodeId, score, gds.util.asNode(nodeId) AS node
RETURN 
    node.chunkId AS Name, 
    score as Score,
    labels(node) AS Labels
ORDER BY Score DESC, Name ASC"""
).head(10)

betweenness_stream

In [ ]:
plot_top_nodes(betweenness_stream, title="Top 10 Nodes by Betweenness Centrality")

# Stats
In the stats execution mode, the algorithm returns a single row containing a summary of the algorithm result. In particular, Betweenness Centrality returns the minimum, maximum and sum of all centrality scores.

In [ ]:
betweenness_stats = gds.run_cypher(
    """CALL gds.betweenness.stats('myGraph')
YIELD centralityDistribution
RETURN centralityDistribution.min AS MinimumScore, centralityDistribution.mean AS MeanScore"""
)

betweenness_stats

In [ ]:
betweenness_stats.to_csv(r"betweenness_stats.csv", index=False, encoding="utf-8")

# Influence Maximization (CELF)
The influence maximization problem asks for a set of k nodes that maximize the expected spread of influence in the network. The set of these initial k is called the seed set.

The Neo4j GDS Library supports approximate computation of the best seed set under the Independent Cascade propagation model. In this propagation model, initially we assume that the nodes in the seed set become influenced and the process works as follows. An influenced node influences each of its neighbors with probability p. The spread is then the number of nodes that become influenced.

# Stream
In the stream execution mode, the algorithm returns the spread for nodes that are part of the seed set.

In [ ]:
influence_stream = gds.run_cypher(
    """CALL gds.influenceMaximization.celf.stream('myGraph', {seedSetSize: 10})
YIELD nodeId, spread
WITH nodeId, spread, gds.util.asNode(nodeId) AS node
RETURN 
    node.chunkId AS Name, 
    spread as Spread,
    labels(node) AS Labels
ORDER BY spread DESC, Name ASC"""
)

influence_stream

In [ ]:
plot_top_nodes(influence_stream, "Spread", "Top 10 Nodes by Influence Spread")

# Stats
In the stats execution mode, the algorithm returns a single row containing a summary of the algorithm result.

In [ ]:
influence_stats = gds.run_cypher(
    """CALL gds.influenceMaximization.celf.stats('myGraph', {seedSetSize: 10})
YIELD totalSpread as TotalSpread"""
)

influence_stats

In [ ]:
influence_stats.to_csv(r"influence_stats.csv", index=False, encoding="utf-8")

# Closeness Centrality
Closeness centrality is a way of detecting nodes that are able to spread information very efficiently through a graph.

The closeness centrality of a node measures its average farness (inverse distance) to all other nodes. Nodes with a high closeness score have the shortest distances to all other nodes.

For each node u, the Closeness Centrality algorithm calculates the sum of its distances to all other nodes, based on calculating the shortest paths between all pairs of nodes. The resulting sum is then inverted to determine the closeness centrality score for that node.

# Stream
In the stream execution mode, the algorithm returns the centrality for each node.

In [ ]:
closeness_stream = gds.run_cypher(
    """CALL gds.closeness.stream('myGraph')
YIELD nodeId, score
WITH nodeId, score, gds.util.asNode(nodeId) AS node
RETURN 
    node.chunkId AS Name,
    score as Score,
    labels(node) AS Labels
ORDER BY score DESC, Name ASC"""
).head(10)

closeness_stream

In [ ]:
plot_top_nodes(closeness_stream, title="Top 10 Nodes by Closeness Centrality")

# Stats
In the stats execution mode, the algorithm returns a single row containing a summary of the algorithm result.

In [ ]:
closeness_stats = gds.run_cypher(
    """CALL gds.closeness.stats('myGraph')
YIELD centralityDistribution
RETURN centralityDistribution.min AS MinimumScore, centralityDistribution.mean AS MeanScore"""
)

closeness_stats

In [ ]:
closeness_stats.to_csv(r"closeness_stats.csv", index=False, encoding="utf-8")

# Degree Centrality
The Degree Centrality algorithm can be used to find popular nodes within a graph. The degree centrality measures the number of incoming or outgoing (or both) relationships from a node, which can be defined by the orientation of a relationship projection. If a projection contains directed relationships, only outgoing relationships of a node are counted (the out-degree).

# Stream
In the stream execution mode, the algorithm returns the degree centrality for each node.

In [ ]:
degree_stream = gds.run_cypher(
    """CALL gds.degree.stream('myGraph')
YIELD nodeId, score
WITH nodeId, score, gds.util.asNode(nodeId) AS node
RETURN 
    node.chunkId AS Name,
    score as Score,
    labels(node) AS Labels
ORDER BY score DESC, Name ASC"""
).head(10)

degree_stream

In [ ]:
plot_top_nodes(degree_stream, title="Top 10 Nodes by Degree Centrality")

# Stats
In the stats execution mode, the algorithm returns a single row containing a summary of the algorithm result.

In [ ]:
degree_stats = gds.run_cypher(
    """CALL gds.degree.stats('myGraph')
YIELD centralityDistribution
RETURN centralityDistribution.min AS MinimumScore, centralityDistribution.mean AS MeanScore"""
)

degree_stats

In [ ]:
degree_stats.to_csv(r"degree_stats.csv", index=False, encoding="utf-8")

# Eigenvector Centrality
Eigenvector Centrality is an algorithm that measures the transitive influence of nodes. Relationships originating from high-scoring nodes contribute more to the score of a node than connections from low-scoring nodes. A high eigenvector score means that a node is connected to many nodes who themselves have high scores.

The algorithm computes the eigenvector associated with the largest absolute eigenvalue. To compute that eigenvalue, the algorithm applies the power iteration approach. Within each iteration, the centrality score for each node is derived from the scores of its incoming neighbors. In the power iteration method, the eigenvector is L2-normalized after each iteration, leading to normalized results by default.

The PageRank algorithm is a variant of Eigenvector Centrality with an additional jump probability.

# Stream
In the stream execution mode, the algorithm returns the score for each node.

In [ ]:
eigenvector_stream = gds.run_cypher(
    """CALL gds.eigenvector.stream('myGraph', {maxiterations: 40})
YIELD nodeId, score
WITH nodeId, score, gds.util.asNode(nodeId) AS node
RETURN
    node.chunkId AS Name,
    score as Score,
    labels(node) AS Labels
ORDER BY score DESC, Name ASC"""
).head(10)

eigenvector_stream

In [ ]:
plot_top_nodes(eigenvector_stream, title="Top 10 Nodes by Eigenvector Centrality")

# PageRank
The PageRank algorithm measures the importance of each node within the graph, based on the number of incoming relationships and the importance of the corresponding source nodes. The underlying assumption roughly speaking is that a page is only as important as the pages that link to it.

# Stream
In the stream execution mode, the algorithm returns the score for each node.

In [ ]:
pagerank_stream = gds.run_cypher(
    """CALL gds.pageRank.stream('myGraph', {maxiterations: 40})
YIELD nodeId, score
WITH nodeId, score, gds.util.asNode(nodeId) AS node
RETURN
    node.chunkId AS Name,
    score as Score,
    labels(node) AS Labels
ORDER BY score DESC, Name ASC"""
).head(10)

pagerank_stream

In [ ]:
plot_top_nodes(pagerank_stream, title="Top 10 Nodes by PageRank")

# Harmonic Centrality
Harmonic centrality was proposed by Marchiori and Latora in Harmony in the Small World while trying to come up with a sensible notion of "average shortest path".

They suggested a different way of calculating the average distance to that used in the Closeness Centrality algorithm. Rather than summing the distances of a node to all other nodes, the harmonic centrality algorithm sums the inverse of those distances. This enables it deal with infinite values.

# Stream
In the stream execution mode, the algorithm returns the score for each node.

In [ ]:
harmonic_stream = gds.run_cypher(
    """CALL gds.closeness.harmonic.stream('myGraph')
YIELD nodeId, score
WITH nodeId, score, gds.util.asNode(nodeId) AS node
RETURN
    node.chunkId AS Name,
    score as Score,
    labels(node) AS Labels
ORDER BY score DESC, Name ASC"""
).head(10)

harmonic_stream

In [ ]:
plot_top_nodes(harmonic_stream, title="Top 10 Nodes by Harmonic Centrality")